In [224]:

date='''"2022-01-21 00:00:00.000000"'''

import pyomo.environ as pyo
# install glpk in pyomo/solvers/ folder.
import pandas as pd

import sqlite3

cnx = sqlite3.connect('/Users/kultiginbozdemir/Documents/GitHub/op_room_opt/WebApp/test2.db') # change the path

class Solve_Problem:

    def get_ops(self,date):
        query = ''.join(['SELECT * FROM user__entries WHERE operation_date==',str(date)])# change name of the table.  save them with date in db. 

        basket = pd.read_sql(query,cnx)
        basket.index=basket.id

        query_dep_cap =''.join(['SELECT * FROM department__info WHERE date==',str(date)]) # table of department capacities
        dep_df = pd.read_sql_query(query_dep_cap, cnx)# departments
        dep_df.index=dep_df.department_name
        dep_df['department_capacity']=dep_df['department_capacity'].astype(int)



        query_op_cap = ''.join(['SELECT * FROM operation_rooms__info WHERE date==',str(date)]) # table of operation room capacities

        op_df = pd.read_sql_query(query_op_cap, cnx)# operations room
        op_df.index=op_df.room_name
        op_df['room_capacity']=op_df['room_capacity'].astype(int)
        
        # multi-knapsack, integer divisible

        mdl = pyo.ConcreteModel()

        # make sure id is also index in database user__entries
        mdl.invs = pyo.Set(initialize=list(zip(basket.index, basket["department_name"])))
        mdl.bins = pyo.Set(initialize=list(op_df.room_name)) ## list of operations room from db
        mdl.deps = pyo.Set(initialize=list(dep_df.department_name)) # list of departments from db

        # params
        mdl.value   = pyo.Param(mdl.invs, initialize= {(i,row["department_name"]):row["operation_urgency"] for i,row in basket.iterrows()} )
        mdl.weight  = pyo.Param(mdl.invs, initialize= {(i,row["department_name"]):row["operation_duration"] for i,row in basket.iterrows()})
        mdl.bin_cap = pyo.Param(mdl.bins, initialize= {i:row["room_capacity"] for i,row in op_df.iterrows()} )
        mdl.dep_cap = pyo.Param(mdl.deps, initialize= {i:row["department_capacity"] for i,row in dep_df.iterrows()}, mutable=True)



        # vars
        mdl.X = pyo.Var(mdl.invs, mdl.bins, within=pyo.Binary)     # the amount from invoice i in bin j



        ### Objective ###

        mdl.OBJ = pyo.Objective(expr=sum(mdl.X[i, b]*mdl.value[i] for 
                                i in mdl.invs for
                                b in mdl.bins), sense=pyo.maximize)


        ### constraints ###

        # don't overstuff bin
        def bin_limit(self, b):
            return sum(mdl.X[i, b]*mdl.weight[i] for i in mdl.invs) <= mdl.bin_cap[b]
        mdl.bin_limit = pyo.Constraint(mdl.bins, rule=bin_limit)

        # one_item can be only in a single op room.
        def one_item(self, i,d):
            return sum(mdl.X[i,d,b] for b in mdl.bins) <=1
        mdl.one_item = pyo.Constraint(mdl.invs, rule=one_item)



        # department limits

        mdl.dep_limits=pyo.ConstraintList()

        for d in mdl.deps:
            d_list=[]
            for i in mdl.X:
                if d==i[1]:
                    d_list.append(i)    
            mdl.dep_limits.add(expr=(sum(mdl.X[i]*mdl.weight[i[:2]] for i in d_list)<=mdl.dep_cap[d])) 



        # solve it...
        solver = pyo.SolverFactory('glpk')
        results = solver.solve(mdl)

        # save the output into dictionary


        dic=dict()

        for i in mdl.X:
            if pyo.value(mdl.X[i])==1:
                dic[i[0]]=i[2]

        #print(dic.keys())
        #print('basket', basket.index)
        if basket.index.isin(list(dic.keys())).all():
            #print(basket.tail(1).index, 'is registered')
            return dic

        else:
            #print(basket.tail(1).index ,'is NOT registered')
            return('Not possible')

In [225]:
a=Solve_Problem()
a.get_ops(date)

{'Department_2-f7cb7a5b-ec5b-4157-84a9-c67a634dae7a': 'Room_2',
 'Department_2-d31b80c6-5205-4781-a40d-3c98856906f9': 'Room_1',
 'Department_2-b02bc3c2-5421-42e7-9594-072172eecd5f': 'Room_5',
 'Department_3-c1bcae1d-bad6-4607-8b91-196c2d935977': 'Room_1',
 'Department_5-121d22d5-c97d-47d5-8948-f41431f7ab21': 'Room_3',
 'Department_1-daa314dd-52a5-47ae-964c-0df1f10687d1': 'Room_1',
 'Department_4-cdb011f8-5e28-48bf-9055-913b066db356': 'Room_4',
 'Department_3-64aef08a-2707-41ac-b1ae-95833498562f': 'Room_3'}